# 関数呼び出し（Function Calling）

関数呼び出しは、LLMが最終的な回答を生成する際にツールの支援が必要な場合に有用です。

現在のLLMはツール呼び出し機能を備えており、LLMは最終的な回答に支援が必要な場合にツールを呼び出すことができます。

In [1]:
# 警告のインポート
import warnings
warnings.filterwarnings("ignore")

from utils import *
import json

ダミー関数を定義：

In [2]:
# 現在の天気を取得するダミー関数を定義
def get_current_weather(location, unit="fahrenheit"):
    """指定された場所の現在の天気を取得する"""
    weather = {
        "location": location,
        "temperature": "50",
        "unit": unit,
    }

    return json.dumps(weather)

### 関数の定義

OpenAIのドキュメントで示されているように、リクエストの一部となる関数を定義する簡単な例を示します。

説明は重要です。これらは`system message`の一部としてLLMに直接渡され、LLMは説明を使用して関数を使用するかどうかを判断します。

In [3]:
# ツールとして関数を定義
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "指定された場所の現在の天気を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "都市と州、例：San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

### 関数呼び出しのベストプラクティス

- 関数名、説明、パラメータは明確に記述する
- 関数はコンテキスト制限にカウントされるため、関数の数と説明の長さに注意する
- 既に知っている引数をモデルに生成させない
- 精度向上のために関数の数を少なく保つ（OpenAIは約20個を推奨）
- 多数の関数がある場合は、精度向上とコスト削減（トークン使用量の節約）のためにファインチューニングオプションを検討する

それではテストしてみましょう：

In [4]:
# メッセージのリストを定義

messages = [
    {
        "role": "user",
        "content": "ロンドンの天気はどうですか？"
    }
]

In [5]:
response = get_chat_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_icqvcE7zqqUpt2yGLPX03X7p', function=Function(arguments='{"location":"London, UK"}', name='get_current_weather'), type='function')])


引数を取得できます：



In [6]:
args = json.loads(response.tool_calls[0].function.arguments)
print(args)

{'location': 'London, UK'}


実際の関数/ツールに引数を渡す：

In [7]:
get_current_weather(**args)

'{"location": "London, UK", "temperature": "50", "unit": "fahrenheit"}'

### 関数呼び出しの動作制御

LLM駆動の会話エージェントのコンテキストでこの`function_calling`機能を設計することに興味があるとしましょう。あなたのソリューションは、どの関数を呼び出すか、または呼び出す必要があるかどうかを知る必要があります。挨拶メッセージの簡単な例を試してみましょう：

In [8]:
messages = [
    {
        "role": "user",
        "content": "こんにちは！お元気ですか？",
    }
]

In [9]:
get_chat_completion(messages, tools=tools)

ChatCompletionMessage(content="Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

関数呼び出しから期待する動作を指定できます。これは、システムの動作を制御するために望ましいことです。デフォルトでは、モデルは独自に関数を呼び出すかどうか、どの関数を呼び出すかを決定します。これは`tool_choice: "auto"`を設定することで実現されます。これは`default`設定です。

In [10]:
get_chat_completion(messages, tools=tools, tool_choice="auto")


ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

`tool_choice: "none"`を設定すると、モデルは提供された関数のいずれも使用しないように強制されます。

In [11]:
get_chat_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

ユーザーメッセージでテスト：

In [12]:
messages = [
    {
        "role": "user",
        "content": "ロンドンの天気はどうですか？",
    }
]
get_chat_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='Would you like the temperature in Celsius or Fahrenheit?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

アプリケーションで望む動作がある場合は、モデルに関数を選択するように強制することもできます。例：

In [13]:
messages = [
    {
        "role": "user",
        "content": "ロンドンの天気はどうですか？",
    }
]
get_chat_completion(messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_current_weather"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3nx7XsVPO1OfPo2n3KuEjoZK', function=Function(arguments='{"location":"London, UK"}', name='get_current_weather'), type='function')])

OpenAI APIは、1回のターンで複数の関数を呼び出す並列関数呼び出しもサポートしています。効率性の向上と、抽出したい異なる引数セットがある場合に役立ちます。

In [21]:
messages = [
    {
        "role": "user",
        "content": "ロンドンとベルモパンの天気はどうですか？",
    }
]
get_chat_completion(messages, tools=tools)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Xbvyc6IWOp7v9WPwQuy2l9fG', function=Function(arguments='{"location": "London"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_rGRS88ukRw8EzwTSx1X3oELR', function=Function(arguments='{"location": "Belmopan"}', name='get_current_weather'), type='function')])

上記の応答で、照会された2つの場所の関数呼び出しからの情報が含まれていることがわかります。



### モデルフィードバックのための関数呼び出し応答
関数呼び出しから生成された入力でAPIを呼び出した後に得られた結果を渡すエージェントの開発にも興味があるかもしれません。次に例を見てみましょう：

In [23]:
messages = []
messages.append({"role": "user", "content": "ボストンの天気はどうですか！"})
assistant_message = get_chat_completion(messages, tools=tools, tool_choice="auto")
assistant_message = json.loads(assistant_message.model_dump_json())
assistant_message["content"] = str(assistant_message["tool_calls"][0]["function"])

del assistant_message["function_call"]

In [24]:
print(assistant_message)

{'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}', 'refusal': None, 'role': 'assistant', 'audio': None, 'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k', 'function': {'arguments': '{"location":"Boston, MA"}', 'name': 'get_current_weather'}, 'type': 'function'}]}


In [26]:
print(messages)

[{'role': 'user', 'content': "What's the weather like in Boston!"}]


In [27]:
messages.append(assistant_message)

In [28]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]}]

次に、`get_current_weather`関数の結果を追加し、`tool`ロールを使用してモデルに戻します。


In [29]:
# モデルに戻す天気情報を取得
weather = get_current_weather(messages[1]["tool_calls"][0]["function"]["arguments"])

messages.append({"role": "tool",
                 "tool_call_id": assistant_message["tool_calls"][0]["id"],
                 "name": assistant_message["tool_calls"][0]["function"]["name"],
                 "content": weather})

In [30]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston!"},
 {'content': '{\'arguments\': \'{"location":"Boston, MA"}\', \'name\': \'get_current_weather\'}',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'tool_calls': [{'id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
    'function': {'arguments': '{"location":"Boston, MA"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_HjEsBatvMQLfyvpW3YjaRd6k',
  'name': 'get_current_weather',
  'content': '{"location": "{\\"location\\":\\"Boston, MA\\"}", "temperature": "50", "unit": "fahrenheit"}'}]

基本的に、関数から取得した情報をリクエストの一部としてモデルに注入し直しました。モデルは今、そのコンテキストをすべて使用して適切な応答を生成できます。関数に戻り値がない場合（例：`send_email()`）は、成功文字列（例：`"success"`）を返すだけです。

In [31]:
final_response = get_chat_completion(messages, tools=tools)
print(final_response)

ChatCompletionMessage(content='The current temperature in Boston, MA is 50°F.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
